In [177]:
!pip install snowflake-sqlalchemy

In [178]:
import os
from sqlalchemy import create_engine, text
import pandas as pd
from dotenv import load_dotenv

load_dotenv() 

# Obtener credenciales desde variables de entorno
USER = os.getenv("SNOWFLAKE_USER")
PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
DATABASE = os.getenv("SNOWFLAKE_DATABASE")
SCHEMA = os.getenv("SNOWFLAKE_SCHEMA")
WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")

print(f"USER: {USER}")
print(f"ACCOUNT: {ACCOUNT}")
print(f"DATABASE: {DATABASE}")
print(f"SCHEMA: {SCHEMA}")


# Crear la conexión usando SQLAlchemy
engine = create_engine(
    f"snowflake://{USER}:{PASSWORD}@{ACCOUNT}/{DATABASE}/{SCHEMA}?warehouse={WAREHOUSE}"
)

# Verificar conexión
try:
    with engine.connect() as conn:
        print("Conexión exitosa a Snowflake!")
except Exception as e:
    print(f"Error de conexión: {e}")


USER: Lucianav
ACCOUNT: fyqjonx-ro02327
DATABASE: INSTACART_DB
SCHEMA: RAW
Conexión exitosa a Snowflake!


In [179]:
# Obtener las tablas del schema con query
query_tablas = "SHOW TABLES"
df_tablas = pd.read_sql(query_tablas, engine)

table_names = df_tablas["name"].tolist()
print("Tablas encontradas:", table_names)


Tablas encontradas: ['AISLES', 'DEPARTMENTS', 'INSTACART_ORDERS', 'ORDER_PRODUCTS', 'PRODUCTS']


In [180]:
# Extraer datos y crear tablas en dataframes

dataframes = {}  

for table in table_names:
    query = f"SELECT * FROM {DATABASE}.{SCHEMA}.{table}"
    df = pd.read_sql(query, engine)
    dataframes[table] = df
    print(f"Tabla {table} cargada con {len(df)} filas.")


Tabla AISLES cargada con 134 filas.
Tabla DEPARTMENTS cargada con 21 filas.
Tabla INSTACART_ORDERS cargada con 478967 filas.
Tabla ORDER_PRODUCTS cargada con 4545007 filas.
Tabla PRODUCTS cargada con 49694 filas.


In [181]:
print(dataframes.keys())

dict_keys(['AISLES', 'DEPARTMENTS', 'INSTACART_ORDERS', 'ORDER_PRODUCTS', 'PRODUCTS'])


AISLES

In [182]:
aisles_df = dataframes["AISLES"] 
aisles_df

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [183]:
col = aisles_df.columns
rows = aisles_df.shape[0]
print(f"Columnas: {col}")
print(f"Filas: {rows}")

Columnas: Index(['aisle_id', 'aisle'], dtype='object')
Filas: 134


In [184]:
# Estadisticas descriptivas
stats = aisles_df.describe(include='all')
print(stats)

          aisle_id                  aisle
count   134.000000                    134
unique         NaN                    134
top            NaN  prepared soups salads
freq           NaN                      1
mean     67.500000                    NaN
std      38.826537                    NaN
min       1.000000                    NaN
25%      34.250000                    NaN
50%      67.500000                    NaN
75%     100.750000                    NaN
max     134.000000                    NaN


In [185]:
# valores unicos
unique_values = aisles_df.nunique()
print(unique_values)

aisle_id    134
aisle       134
dtype: int64


In [186]:
# Analisis de calidad de datos
missing_values = aisles_df.isnull().sum()
# verificar duplicados
duplicates = aisles_df.duplicated().sum()
print(f"Total valores faltantes: {missing_values.sum()}")
print(f"Total duplicados: {duplicates}")

Total valores faltantes: 0
Total duplicados: 0


DEPARTMENTS

In [187]:
departments_df = dataframes["DEPARTMENTS"] 
departments_df

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [188]:
col = departments_df.columns
rows = departments_df.shape[0]
print(f"Columnas: {col}")
print(f"Filas: {rows}")

Columnas: Index(['department_id', 'department'], dtype='object')
Filas: 21


In [189]:
# Estadisticas descriptivas
stats = departments_df.describe(include='all')
print(stats)

        department_id department
count       21.000000         21
unique            NaN         21
top               NaN     frozen
freq              NaN          1
mean        11.000000        NaN
std          6.204837        NaN
min          1.000000        NaN
25%          6.000000        NaN
50%         11.000000        NaN
75%         16.000000        NaN
max         21.000000        NaN


In [190]:
# valores unicos
unique_values = departments_df.nunique()
print(unique_values)

department_id    21
department       21
dtype: int64


In [191]:
# Analisis de calidad de datos
missing_values = departments_df.isnull().sum()
# verificar duplicados
duplicates = departments_df.duplicated().sum()
print(f"Total valores faltantes: {missing_values.sum()}")
print(f"Total duplicados: {duplicates}")

Total valores faltantes: 0
Total duplicados: 0


INSTACART_ORDERS

In [192]:
instacart_orders_df = dataframes["INSTACART_ORDERS"]
instacart_orders_df

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2788788,188386,18,6,10,3.0
1,665595,124269,54,0,10,19.0
2,160129,12655,4,4,15,29.0
3,2992628,201211,1,6,10,NaN
4,3360760,59786,17,2,17,0.0
...,...,...,...,...,...,...
478962,1397791,133217,32,5,10,4.0
478963,1068973,78030,14,0,14,14.0
478964,2852788,34333,3,1,16,30.0
478965,634245,147019,3,2,12,30.0


In [193]:
col = instacart_orders_df.columns
rows = instacart_orders_df.shape[0]
print(f"Columnas: {col}")
print(f"Filas: {rows}")
# Estadisticas descriptivas
stats = instacart_orders_df.describe(include='all')
print(stats)
# valores unicos
unique_values = instacart_orders_df.nunique()
print(unique_values)
# Analisis de calidad de datos
missing_values = instacart_orders_df.isnull().sum()
# verificar duplicados
duplicates = instacart_orders_df.duplicated().sum()
print(f"Total valores faltantes: {missing_values.sum()}")
print(f"Total duplicados: {duplicates}")

Columnas: Index(['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')
Filas: 478967
           order_id        user_id   order_number      order_dow  \
count  4.789670e+05  478967.000000  478967.000000  478967.000000   
mean   1.709947e+06  102966.537475      17.157808       2.775058   
std    9.869480e+05   59525.328991      17.704688       2.045869   
min    4.000000e+00       2.000000       1.000000       0.000000   
25%    8.564685e+05   51414.000000       5.000000       1.000000   
50%    1.709522e+06  102633.000000      11.000000       3.000000   
75%    2.562990e+06  154419.000000      23.000000       5.000000   
max    3.421079e+06  206209.000000     100.000000       6.000000   

       order_hour_of_day  days_since_prior_order  
count      478967.000000           450148.000000  
mean           13.446676               11.101751  
std             4.224986                9.189975  
min             0.00000

In [194]:
instacart_orders_df["order_id"].value_counts()

order_id
391768     2
2232988    2
1286742    2
2282673    2
1919531    2
          ..
1397791    1
1068973    1
2852788    1
634245     1
400106     1
Name: count, Length: 478952, dtype: int64

ORDER_PRODUCTS

In [195]:
order_products_df = dataframes["ORDER_PRODUCTS"]
order_products_df

,order_id,product_id,add_to_cart_order,reordered
0,2141543,11440,17.0,0
1,567889,1560,1.0,1
2,2261212,26683,1.0,1
3,491251,8670,35.0,1
4,2571142,1940,5.0,1
...,...,...,...,...
4545002,1595287,21903,6.0,1
4545003,169741,21292,7.0,1
4545004,3315620,35199,9.0,1
4545005,1290233,42557,1.0,1


In [196]:
col = order_products_df.columns
rows = order_products_df.shape[0]
print(f"Columnas: {col}")
print(f"Filas: {rows}")
# Estadisticas descriptivas
stats = order_products_df.describe(include='all')
print(stats)
# valores unicos
unique_values = order_products_df.nunique()
print(unique_values)
# Analisis de calidad de datos
missing_values = order_products_df.isnull().sum()
# verificar duplicados
duplicates = order_products_df.duplicated().sum()
print(f"Total valores faltantes: {missing_values.sum()}")
print(f"Total duplicados: {duplicates}")

Columnas: Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Filas: 4545007
           order_id    product_id  add_to_cart_order     reordered
count  4.545007e+06  4.545007e+06       4.544171e+06  4.545007e+06
mean   1.711166e+06  2.558084e+04       8.351669e+00  5.905025e-01
std    9.850955e+05  1.409552e+04       7.080368e+00  4.917411e-01
min    4.000000e+00  1.000000e+00       1.000000e+00  0.000000e+00
25%    8.608170e+05  1.353500e+04       3.000000e+00  0.000000e+00
50%    1.711530e+06  2.527200e+04       6.000000e+00  1.000000e+00
75%    2.561311e+06  3.793500e+04       1.100000e+01  1.000000e+00
max    3.421079e+06  4.969400e+04       6.400000e+01  1.000000e+00
order_id             450046
product_id            45573
add_to_cart_order        64
reordered                 2
dtype: int64
Total valores faltantes: 836
Total duplicados: 0


In [197]:
order_products_df["order_id"].value_counts()

order_id
61355      127
3308010    115
2136777    108
171934     104
1959075     98
          ... 
2824933      1
1844226      1
3254310      1
2396177      1
3039787      1
Name: count, Length: 450046, dtype: int64

In [198]:
# verificar valores nulos
order_products_df.isnull().sum()

order_id               0
product_id             0
add_to_cart_order    836
reordered              0
dtype: int64

In [199]:
order_products_df["add_to_cart_order"].value_counts()

add_to_cart_order
1.0     450046
2.0     428199
3.0     401907
4.0     372861
5.0     341807
         ...  
60.0       127
61.0       114
62.0        98
63.0        87
64.0        75
Name: count, Length: 64, dtype: int64

PRODUCTS

In [200]:
products_df = dataframes["PRODUCTS"]
products_df

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49689,49690,HIGH PERFORMANCE ENERGY DRINK,64,7
49690,49691,ORIGINAL PANCAKE & WAFFLE MIX,130,14
49691,49692,ORGANIC INSTANT OATMEAL LIGHT MAPLE BROWN SUGAR,130,14
49692,49693,SPRING WATER BODY WASH,127,11


In [201]:
col = products_df.columns
rows = products_df.shape[0]
print(f"Columnas: {col}")
print(f"Filas: {rows}")
# Estadisticas descriptivas
stats = products_df.describe(include='all')
print(stats)
# valores unicos
unique_values = products_df.nunique()
print(unique_values)
# Analisis de calidad de datos
missing_values = products_df.isnull().sum()
# verificar duplicados
duplicates = products_df.duplicated().sum()
print(f"Total valores faltantes: {missing_values.sum()}")
print(f"Total duplicados: {duplicates}")

Columnas: Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')
Filas: 49694
          product_id             product_name      aisle_id  department_id
count   49694.000000                    48436  49694.000000   49694.000000
unique           NaN                    48436           NaN            NaN
top              NaN  BURRITO- STEAK & CHEESE           NaN            NaN
freq             NaN                        1           NaN            NaN
mean    24847.500000                      NaN     67.773192      11.728478
std     14345.566475                      NaN     38.317265       5.850314
min         1.000000                      NaN      1.000000       1.000000
25%     12424.250000                      NaN     35.000000       7.000000
50%     24847.500000                      NaN     69.000000      13.000000
75%     37270.750000                      NaN    100.000000      17.000000
max     49694.000000                      NaN    134.000000      21.0

In [202]:
products_df.isnull().sum()

product_id          0
product_name     1258
aisle_id            0
department_id       0
dtype: int64

TRANSFORMACION

In [203]:
# manejo de valores nulos

#promedio de dias entre ordenes
days_mean = instacart_orders_df["days_since_prior_order"].mean()
instacart_orders_df["days_since_prior_order"].fillna(days_mean, inplace=True)
instacart_orders_df.isnull().sum()

/var/folders/3f/k0x4vxjx013cr_5103f5wh480000gn/T/ipykernel_19859/265501512.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  instacart_orders_df["days_since_prior_order"].fillna(days_mean, inplace=True)


order_id                  0
user_id                   0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
dtype: int64

In [204]:
# llenar valores nulos de add_to_cart_order con -1
order_products_df["add_to_cart_order"].fillna(-1, inplace=True)
order_products_df.isnull().sum()

/var/folders/3f/k0x4vxjx013cr_5103f5wh480000gn/T/ipykernel_19859/1881630526.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  order_products_df["add_to_cart_order"].fillna(-1, inplace=True)


order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

STAR SCHEMA

In [212]:
dim_aisles = aisles_df.copy()
dim_products = products_df.copy()
dim_departments = departments_df.copy()
dim_orders = instacart_orders_df.drop_duplicates(subset=["order_id"]).copy()
dim_users = dim_orders[["user_id"]].copy()


instant = instacart_orders_df[["order_id", "user_id","order_dow", "order_hour_of_day", "days_since_prior_order"]]

#merge de tablas usando order_id como key
fact_orders = pd.merge(dim_orders, order_products_df,on="order_id", how="left")

tablas_clean = {"DIM_PRODUCTS": dim_products, "DIM_USERS": dim_users , "DIM_AISLES": dim_aisles, "DIM_DEPARTMENTS": dim_departments, "DIM_ORDERS": dim_orders}

In [213]:
instant["order_id"].value_counts()

order_id
391768     2
2232988    2
1286742    2
2282673    2
1919531    2
          ..
1397791    1
1068973    1
2852788    1
634245     1
400106     1
Name: count, Length: 478952, dtype: int64

In [214]:
fact_orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2788788,188386,18,6,10,3.0,23405.0,4.0,0.0
1,2788788,188386,18,6,10,3.0,1278.0,3.0,0.0
2,2788788,188386,18,6,10,3.0,38662.0,1.0,0.0
3,2788788,188386,18,6,10,3.0,436.0,2.0,1.0
4,665595,124269,54,0,10,19.0,22935.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...
4573908,1520727,191308,13,0,11,16.0,14536.0,12.0,1.0
4573909,1520727,191308,13,0,11,16.0,4149.0,11.0,0.0
4573910,1520727,191308,13,0,11,16.0,26209.0,9.0,0.0
4573911,1520727,191308,13,0,11,16.0,38033.0,14.0,0.0


In [215]:
fact_orders["order_id"].value_counts()

order_id
61355      127
3308010    115
2136777    108
171934     104
1959075     98
          ... 
1461833      1
1496098      1
1468577      1
233728       1
666423       1
Name: count, Length: 478952, dtype: int64

In [216]:
fact_orders["add_to_cart_order"].value_counts()

add_to_cart_order
1.0     450046
2.0     428199
3.0     401907
4.0     372861
5.0     341807
         ...  
60.0       127
61.0       114
62.0        98
63.0        87
64.0        75
Name: count, Length: 65, dtype: int64

In [217]:
dim_orders["order_id"].value_counts()

order_id
1520727    1
2788788    1
665595     1
160129     1
2992628    1
          ..
1558904    1
3282387    1
2523839    1
2995593    1
2828007    1
Name: count, Length: 478952, dtype: int64

CARGAR DATOS A SNOWFLAKE

In [218]:
from sqlalchemy import text
# Crear la conexión con SQLAlchemy
engine = create_engine(
    f'snowflake://{USER}:{PASSWORD}@{ACCOUNT}/{DATABASE}/CLEAN?warehouse={WAREHOUSE}'
)

# Definir el tamaño del batch
batch_size = 10000

# Verificar conexión
try:
    with engine.connect() as conn:
        print("Conexión exitosa a Snowflake!")
except Exception as e:
    print(f"Error de conexión: {e}")
    

# Crear esquema CLEAN si no existe
with engine.connect() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS CLEAN;"))
    print("Esquema 'CLEAN' creado o ya existente.")

# Cargar cada DataFrame en Snowflake con chunksize
for table_name, df in tablas_clean.items():
    df.to_sql(
        table_name,
        con=engine,
        schema="CLEAN",
        index=False,
        if_exists="replace",  # Reemplaza la tabla si ya existe
        chunksize=batch_size,  # Carga en lotes
    )
    print(f"Tabla '{table_name}' cargada en lotes de {batch_size} filas.")

print("Carga de datos en Snowflake completada. ✅")

Conexión exitosa a Snowflake!
Esquema 'CLEAN' creado o ya existente.


/var/folders/3f/k0x4vxjx013cr_5103f5wh480000gn/T/ipykernel_19859/2748351406.py:25: UserWarning: The provided table name 'DIM_PRODUCTS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(


Tabla 'DIM_PRODUCTS' cargada en lotes de 10000 filas.


/var/folders/3f/k0x4vxjx013cr_5103f5wh480000gn/T/ipykernel_19859/2748351406.py:25: UserWarning: The provided table name 'DIM_USERS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(


Tabla 'DIM_USERS' cargada en lotes de 10000 filas.


...

Tabla 'DIM_AISLES' cargada en lotes de 10000 filas.


/var/folders/3f/k0x4vxjx013cr_5103f5wh480000gn/T/ipykernel_19859/2748351406.py:25: UserWarning: The provided table name 'DIM_DEPARTMENTS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(


Tabla 'DIM_DEPARTMENTS' cargada en lotes de 10000 filas.


/var/folders/3f/k0x4vxjx013cr_5103f5wh480000gn/T/ipykernel_19859/2748351406.py:25: UserWarning: The provided table name 'DIM_ORDERS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(


Tabla 'DIM_ORDERS' cargada en lotes de 10000 filas.
Carga de datos en Snowflake completada. ✅


In [219]:
# exportar tabla facts orders csv
fact_orders.to_csv('fact_orders.csv', index=False)